Task 5 – Pivot Table + Stack/Unstack:

1. Tạo pivot tổng số liều tiêm theo nhóm tuổi × vaccine_type:

In [3]:
import pandas as pd
# Đọc dữ liệu từ file CSV:
vaccination = pd.read_csv(r"../data_clean/vaccination_record_clean.csv")
patient = pd.read_csv(r"../data_clean/patient_info_clean.csv")

# 1. Chuẩn hóa patient_id:
vaccination['patient_id'] = vaccination['patient_id'].str.upper().str.strip()
patient['patient_id'] = patient['patient_id'].str.upper().str.strip()

# 2. Chuẩn hóa nhóm tuổi:
patient['age_group'] = (
    patient['age_group']
    .str.strip()
    .str.replace('_', '-', regex=False)
    .replace({
        '50plus': '50+',
        '50 Plus': '50+'
    })
)

# 3. Chuẩn hóa vaccine_type:
vaccination['vaccine_type'] = (
    vaccination['vaccine_type']
    .str.strip()
    .str.lower()
    .replace({
        'covid': 'covid-19',
        'covid19': 'covid-19',
        'd.p.t': 'dpt'
    })
    .str.upper()
)

# 4. Merge dữ liệu:
df = vaccination.merge(
    patient[['patient_id', 'age_group']],
    on='patient_id',
    how='left'
)

# 5. Tạo pivot table:
sum_of_doses = pd.pivot_table(
    df,
    index='age_group',
    columns='vaccine_type',
    values='dose_number',
    aggfunc='count',
    fill_value=0
)
# Sắp xếp thứ tự nhóm tuổi:
age_sort = ['0-5', '6-17', '18-49', '50+']
pivot_sum_doses = sum_of_doses.reindex(age_sort)
print(" Bảng pivot tổng số liều tiêm theo nhóm tuổi × vaccine_type: ")
print(pivot_sum_doses)


 Bảng pivot tổng số liều tiêm theo nhóm tuổi × vaccine_type: 
vaccine_type  COVID-19  DPT  MMR
age_group                       
0-5                  1    3    3
6-17                 8    2    7
18-49                7    6    4
50+                  4    3    3


2. Tạo pivot mức độ phản ứng trung bình theo vaccine_type × age_group:

In [ ]:
import pandas as pd
import numpy as np
import re
# Đọc dữ liệu từ file CSV:
reaction = pd.read_csv("../data_clean/reaction_log_clean.csv")

# 1. Chuẩn hóa patient_id
reaction['patient_id'] = (
    reaction['patient_id']
    .astype(str)
    .str.upper()
    .str.strip()
)

# 2. Làm sạch severity:
def clean_severity(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    if x == 'severe':
        return 3
# Lấy số đầu tiên nếu có:
    m = re.search(r'\d', x)
    if m:
        return int(m.group())
    return np.nan
reaction['severity_clean'] = reaction['severity'].apply(clean_severity)

# 3. Merge dữ liệu:
summary_df = (
    reaction
    .merge(vaccination[['patient_id', 'vaccine_type']], on='patient_id', how='left')
    .merge(patient[['patient_id', 'age_group']], on='patient_id', how='left')
)

# 4. Pivot table:
average_reaction_level = pd.pivot_table(
    summary_df,
    index='vaccine_type',
    columns='age_group',
    values='severity_clean',
    aggfunc='mean'
)
# Sắp xếp thứ tự nhóm tuổi:
age_sort = ['0-5', '6-17', '18-49', '50+']
pivot_avg_reaction = average_reaction_level.reindex(columns=age_sort)
print("\n Bảng pivot mức độ phản ứng trung bình theo vaccine_type × age_group: ")
print(pivot_avg_reaction.round(2))



